# 1. TimeStamp, Caption, Label 불러오기

In [264]:
import numpy as np
import pandas as pd

textdata = pd.read_csv("./input_text/script.csv")

### 텍스트, 라벨 -----------------------------
Labels, Texts= [], []
for T, L in zip(textdata["Text"], textdata["Label"]) :
    Texts.append(T)
    Labels.append(L)


### 타임스탬프 -----------------------------
textdataList = textdata.values.tolist()

start_times, end_times = [], []
for times in textdataList :
    start_times.append(times[0])
    end_times.append(times[1])

### 타임스탬프 -> 초 단위로 변환 --------------
def convert_to_seconds(time):
    h, m, s, ms = map(int, time.split(':'))
    return 3600 * h + 60 * m + s + ms / 100.0

start_times = [convert_to_seconds(time) for time in start_times]
end_times = [convert_to_seconds(time) for time in end_times]

print(len(start_times), len(end_times))


21 21


In [244]:
print(Texts[:3], Labels[:3])

['아, 같이 타자아아아~~  어?', '엄마 아빠도 안 탄다는데... 혼자 타면 재미없단 말이야.....', '아, 됐어. 토 쏠리는 걸 내가 왜 타.'] ['happiness', 'sadness', 'neutral']


In [219]:
### read csv --------------
import numpy as np
import pandas as pd

textdata = pd.read_csv("./input_text/test2.csv")
textdata

,Start Time,End Time,Text,Label
0,00:00:00:12,00:00:02:12,"아, 같이 타자아아아~~ 어?",happiness
1,00:00:02:12,00:00:04:13,엄마 아빠도 안 탄다는데... 혼자 타면 재미없단 말이야.....,sadness
2,00:00:05:00,00:00:07:00,"아, 됐어. 토 쏠리는 걸 내가 왜 타.",neutral
3,00:00:07:00,00:00:09:01,같이 타주면 내가 평생 충성할게~ 응~~?,happiness
4,00:00:09:01,00:00:10:02,응응~~??,happiness
5,00:00:10:02,00:00:11:16,아 오빠아아아~~!,happiness
6,00:00:11:16,00:00:13:01,"참, 알았어.",neutral
7,00:00:13:24,00:00:15:01,"헤헤, 역시 우리 오빠 짱~!",happiness
8,00:00:15:01,00:00:16:08,앞으로 평생 충성하겠습니다~~!,happiness
9,00:00:16:08,00:00:17:26,충성~~!,happiness


In [187]:
set(textdata["Label"])
# 우선 4개의 자막스타일 생성

{'angry', 'happiness', 'neutral', 'sadness', 'surprise'}

In [197]:
### 자막 --------------
texts= []
for i in textdata["Text"] : texts.append(i)
print(len(texts))

22


In [198]:
Labels =[]
for i in textdata["Label"] : Labels.append(i)
print(Labels[:3])

['happiness', 'sadness', 'neutral']


In [189]:
### 타임스탬프 --------------
textdataList = textdata.values.tolist()

print(textdataList[0:3])
start_times, end_times = [], []
for times in textdataList :
    start_times.append(times[0])
    end_times.append(times[1])

print(len(start_times), len(end_times))

[['00:00:00:12', '00:00:02:12', '아, 같이 타자아아아~~  어?', 'happiness'], ['00:00:02:12', '00:00:04:13', '엄마 아빠도 안 탄다는데  혼자 타면 재미없단 말이야..', 'sadness'], ['00:00:05:00', '00:00:07:00', '아, 됐어. 토 쏠리는 걸 내가 왜 타.', 'neutral']]
21 21


In [190]:
### 타임스탬프 -> 초 단위로 변환 --------------
def convert_to_seconds(time):
    h, m, s, ms = map(int, time.split(':'))
    return 3600 * h + 60 * m + s + ms / 100.0

start_times = [convert_to_seconds(time) for time in start_times]
end_times = [convert_to_seconds(time) for time in end_times]

print(len(start_times))
print(len(end_times))

21
21


# 동영상과 자막 합성

In [265]:
### 정해진 TimeStamp에 따라 자막을 합성하는 함수 ----------------

import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

def add_subtitles_within_time_range(text, Label, input_video_path, start_time, end_time):
    # 비디오 로드
    video = cv2.VideoCapture(input_video_path)

    # Get the dimensions of the video
    frame_width = int(video.get(3))
    frame_height = int(video.get(4))

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"mp4v") 
    out = cv2.VideoWriter("./output_video/output.mp4", fourcc, 30.0, (frame_width, frame_height))

    # Define the text font and color
    if Label == 'neutral':
        font = ImageFont.truetype("./fonts/Cafe24Classictype.ttf", 42)
        color = (255, 255, 255)
    elif Label == 'happiness':
        font = ImageFont.truetype("./fonts/HSSantokki-Regular.ttf", 70)
        color = (255, 255, 255)
    elif Label == 'angry':
        font = ImageFont.truetype("./fonts/BMEuljiro10yearslater.ttf", 70)
        color = (128, 0, 16)
    elif Label == 'sadness':
        font = ImageFont.truetype("./fonts/SF함박눈TTF.ttf", 52)
        color = (255, 255, 255)
    elif Label == 'surprise':
        font = ImageFont.truetype("./fonts/개봉박두체.ttf", 68)
        color = (254, 226, 52)    

    # Read and process each frame of the video
    start_frame = int(start_time * 30)
    end_frame = int(end_time * 30)
    current_frame = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break
        if current_frame >= start_frame and current_frame <= end_frame:
            # Convert the frame to a PIL image
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            
            # Create a draw object and add text to the image
            draw = ImageDraw.Draw(frame)
            text = text
            textwidth, textheight = font.getsize(text)
            text_position = (frame_width / 2 - textwidth / 2, frame_height - textheight - 50)
            draw.text(text_position, text, font=font, fill=color)   
            
            # Convert the PIL image back to a OpenCV frame
            frame = np.array(frame)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            
        # Write the frame to the output video
        out.write(frame)
        current_frame += 1

    # Release the video and VideoWriter objects
    out.release()
    video.release()


In [266]:
import shutil # copy 생성 라이브러리
from tqdm import tqdm
import time
# ./input_video/test2.mp4

input_video_path = "./input_video/test2.mp4"
cnt = 0

for i in tqdm(range(len(Texts)), desc=f'{len(Texts)}개의 자막과 동영상을 합성 중 입니다.'):
    cnt += 1
    text = Texts[i]
    start_time = start_times[i]
    end_time = end_times[i]
    Label = Labels[i]
    if cnt == 1 :
        add_subtitles_within_time_range(text, Label, input_video_path, start_time, end_time)
        input_file = "./output_video/output.mp4"
        output_file = "./output_video/final.mp4"
        shutil.copy(input_file, output_file)

    else :
        input_video_path = "./output_video/final.mp4"
        add_subtitles_within_time_range(text, Label, input_video_path, start_time, end_time)
        input_file = "./output_video/output.mp4"
        output_file = "./output_video/final.mp4"
        shutil.copy(input_file, output_file)


21개의 자막과 동영상을 합성 중 입니다.: 100%|██████████| 21/21 [04:29<00:00, 12.84s/it]


# 동영상과 오디오 재합성

In [1]:
# 원본 테스트 영상에서 오디오 추출
import moviepy.editor as mp

clip = mp.VideoFileClip("./input_video/test2.mp4")
clip.audio.write_audiofile("./input_video/audio.mp3")

MoviePy - Writing audio in ./input_video/audio.mp3


MoviePy - Done.


In [3]:
# 합성영상과 오디오 재합성
import subprocess
import os

video_file = "./output_video/final.mp4"
audio_file = "./input_video/audio.mp3"
output_file = "./output_video/final_output.mp4"

command = (f"ffmpeg -i {video_file} -i {audio_file} -c:v copy -c:a aac -strict experimental {output_file}")
subprocess.call(command, shell=True)


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.1.2_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --

0